Import Library

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import string
import re
import csv
import nltk
import requests
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

Memuat Dataset

In [2]:
app_reviews_df = pd.read_csv('Ulasan_Aplikasi_Tiktok.csv')
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,4bf715bb-c9c0-4d81-8e2c-3791fe34ca9b,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,ok,5,0,39.8.4,2025-05-14 06:56:52,NaN,NaN,39.8.4
1,abbd86af-67c3-4c26-965c-f4b001554ae5,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,personal,1,0,NaN,2025-05-14 03:01:16,NaN,NaN,NaN
2,b0f98638-ea55-40ba-acf5-9c10677d5860,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Ok,4,0,NaN,2025-05-14 00:18:30,NaN,NaN,NaN
3,ac829602-f43b-4ddd-926d-a6e30345c148,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,ya itu harus profesional,5,0,NaN,2025-05-13 23:03:42,NaN,NaN,NaN
4,48f6e86e-cdf0-436f-bb25-2cf9c60dd450,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat bagus,5,0,NaN,2025-05-13 11:17:58,NaN,NaN,NaN


In [3]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,4bf715bb-c9c0-4d81-8e2c-3791fe34ca9b,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,ok,5,0,39.8.4,2025-05-14 06:56:52,NaN,NaN,39.8.4
1,abbd86af-67c3-4c26-965c-f4b001554ae5,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,personal,1,0,NaN,2025-05-14 03:01:16,NaN,NaN,NaN
2,b0f98638-ea55-40ba-acf5-9c10677d5860,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Ok,4,0,NaN,2025-05-14 00:18:30,NaN,NaN,NaN
3,ac829602-f43b-4ddd-926d-a6e30345c148,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,ya itu harus profesional,5,0,NaN,2025-05-13 23:03:42,NaN,NaN,NaN
4,48f6e86e-cdf0-436f-bb25-2cf9c60dd450,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat bagus,5,0,NaN,2025-05-13 11:17:58,NaN,NaN,NaN


In [4]:
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              9000 non-null   object
 1   userName              9000 non-null   object
 2   userImage             9000 non-null   object
 3   content               9000 non-null   object
 4   score                 9000 non-null   int64 
 5   thumbsUpCount         9000 non-null   int64 
 6   reviewCreatedVersion  5390 non-null   object
 7   at                    9000 non-null   object
 8   replyContent          1 non-null      object
 9   repliedAt             1 non-null      object
 10  appVersion            5390 non-null   object
dtypes: int64(2), object(9)
memory usage: 773.6+ KB


In [7]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace('\n', ' ')
    text = text.strip(' ')
    text = text.lower()
    return text

app_reviews_df['content'] = app_reviews_df['content'].apply(cleaningText)

In [8]:
def casefoldingText(text):
    text = text.lower()
    return text

In [9]:
def tokenizingText(text):
    text = word_tokenize(text)
    return text

In [10]:
def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

In [11]:
def stemmingText(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

In [12]:
def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

In [13]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [14]:
app_reviews_df['text_clean'] = app_reviews_df['content'].apply(cleaningText)
app_reviews_df['text_casefoldingText'] = app_reviews_df['text_clean'].apply(casefoldingText)
app_reviews_df['text_slangwords'] = app_reviews_df['text_casefoldingText'].apply(fix_slangwords)
app_reviews_df['text_tokenizingText'] = app_reviews_df['text_slangwords'].apply(tokenizingText)
app_reviews_df['text_stopword'] = app_reviews_df['text_tokenizingText'].apply(filteringText)
app_reviews_df['text_akhir'] = app_reviews_df['text_stopword'].apply(toSentence)

In [15]:
clean_df = app_reviews_df.drop_duplicates(subset=['text_akhir']).dropna(subset=['text_akhir'])

jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape
print(f"Jumlah ulasan setelah hapus duplikat: {jumlah_ulasan_setelah_hapus_duplikat}")
print(f"Jumlah kolom setelah hapus duplikat: {jumlah_kolom_setelah_hapus_duplikat}")

Jumlah ulasan setelah hapus duplikat: 5182
Jumlah kolom setelah hapus duplikat: 17


Pelabelan Sentimen

In [16]:
lexicon_negative = dict()
response_neg = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

if response_neg.status_code == 200:
    reader_neg = csv.reader(StringIO(response_neg.text), delimiter=',')

    for row in reader_neg:
        lexicon_negative[row[0]] = int(row[1])
else:
    print("Failed to fetch negative lexicon data")

In [17]:
lexicon_positive = dict()
response_pos = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')

if response_pos.status_code == 200:
    reader_pos = csv.reader(StringIO(response_pos.text), delimiter=',')

    for row in reader_pos:
        lexicon_positive[row[0]] = int(row[1])
else:
    print("Failed to fetch positive lexicon data")

In [18]:
def sentiment_analysis_lexicon_indonesia(text):

    score = 0

    if not isinstance(text, str):
        return 'neutral'

    text_tokens = word_tokenize(text)

    for word in text_tokens:
        score += lexicon_positive.get(word, 0)
        score += lexicon_negative.get(word, 0)

    if score > 0:
        return 'positive'
    elif score < 0:
        return 'negative'
    else:
        return 'neutral'

    return score, polarity

In [19]:
clean_df['sentiment'] = clean_df['text_akhir'].apply(sentiment_analysis_lexicon_indonesia)
print(clean_df['sentiment'].value_counts())

sentiment
neutral     2118
negative    1613
positive    1451
Name: count, dtype: int64


Tokenisasi dan Padding

In [20]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")  # Ukuran vocabulary
tokenizer.fit_on_texts(clean_df['text_akhir'])
sequences = tokenizer.texts_to_sequences(clean_df['text_akhir'])
padded = pad_sequences(sequences, maxlen=200, truncating='post')

In [21]:
# One-hot encode label
sentiment_labels = pd.get_dummies(clean_df['sentiment']).values

In [22]:
# membagi data
X_train, X_test, y_train, y_test = train_test_split(padded, sentiment_labels, test_size=0.2, random_state=42)

Modeling

In [23]:
model_lstm = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(5000, 64, input_length=200),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 kelas: positif, negatif, netral
])

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_lstm.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


130/130 ━━━━━━━━━━━━━━━━━━━━ 20s 134ms/step - accuracy: 0.4794 - loss: 0.9821 - val_accuracy: 0.7445 - val_loss: 0.6551
Epoch 2/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 19s 122ms/step - accuracy: 0.8190 - loss: 0.5211 - val_accuracy: 0.8312 - val_loss: 0.4763
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 21s 127ms/step - accuracy: 0.9166 - loss: 0.2633 - val_accuracy: 0.8390 - val_loss: 0.4252
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 20s 122ms/step - accuracy: 0.9599 - loss: 0.1485 - val_accuracy: 0.8573 - val_loss: 0.4310
Epoch 5/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 21s 123ms/step - accuracy: 0.9740 - loss: 0.0973 - val_accuracy: 0.8582 - val_loss: 0.4279
Epoch 6/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 22s 133ms/step - accuracy: 0.9734 - loss: 0.0909 - val_accuracy: 0.8611 - val_loss: 0.4549
Epoch 7/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 20s 130ms/step - accuracy: 0.9823 - loss: 0.0708 - val_accuracy: 0.8631 - val_loss: 0.4997
Epoch 8/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 20s 122ms/step - accuracy: 0.9838 - loss: 0.0603 - val

Evaluasi

In [24]:
loss, accuracy = model_lstm.evaluate(X_test, y_test)
print(f"Akurasi Model LSTM: {accuracy}")

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.8592 - loss: 0.5321
Akurasi Model LSTM: 0.8534233570098877


Inferensi

In [26]:
def predict_sentiment(text):
    text = cleaningText(text)
    text = casefoldingText(text)
    text = fix_slangwords(text)
    text_tokens = tokenizingText(text)
    text_filtered = filteringText(text_tokens)
    text_final = toSentence(text_filtered)

    sequence = tokenizer.texts_to_sequences([text_final])
    padded_seq = pad_sequences(sequence, maxlen=200, truncating='post')
    prediction = model_lstm.predict(padded_seq)
    label_index = np.argmax(prediction)
    labels = ['negative', 'neutral', 'positive']
    return labels[label_index]

In [28]:
new_review = input("Masukkan Kalimat Baru:")
sentiment = predict_sentiment(new_review)
print(f"Sentimen Ulasan: {sentiment}")

Masukkan Kalimat Baru:aplikasi buruk
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Sentimen Ulasan: negative
